In [0]:
# Mount Google Drive for storage
# see https://colab.research.google.com/notebooks/io.ipynb for more info
from google.colab import drive
drive.mount('/g/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /g/


In [0]:
# Autheticate towards gcs bucket
from google.colab import auth
auth.authenticate_user()


In [0]:
!gsutil cp gs://landmark-training/csvs.tar.gz .
!tar -xzvf csvs.tar.gz

In [0]:
!gsutil cp gs://landmark-training/test-images.tar .
!tar -xf test-images.tar

In [0]:
from fastai.vision import *
import fastai
from PIL import Image
from io import BytesIO
import requests
import os

In [0]:
# Google Drive path (to load exported model and to save csv)
path = "/g/My Drive/"
# Path to test.csv
csv_path = Path("/content/test.csv")

In [0]:
# Load test.csv into dataframe
test_df = pd.read_csv(csv_path)

In [0]:
# Create a dataframe for the submissions
submission_df = pd.DataFrame(columns=['id', 'landmarks'])

In [0]:
# ImageList from test images
test_images = ImageList.from_folder('/content/test')        

In [0]:
# load the exported learner (exported from the training notebook)
learn = load_learner(path, "yourmodel.pkl", test=test_images) #.to_fp16()

In [0]:
# set the batch size
learn.data.batch_size = 128

In [0]:
# Get the predictions 
preds, y = learn.get_preds(ds_type=DatasetType.Test)

In [0]:
# Create a dictionary with the "<category/label> <confidence score>" as value 
# and image id (= image filename without file extension) as key 
pred_to_filename = {}
i = 0
for pred in preds:
  filename = os.path.basename(os.path.splitext(test_images.items[i])[0])
  # get the predicted category via the preds tensor
  category = learn.data.classes[preds[i].argmax().item()]
  # and its probabilty 
  probability = preds[i][preds[i].argmax().item()].item()
  # add an element to the dictionary with "<category> <probability>" with 
  # filename as key
  pred_to_filename[filename] = str(category)+" "+str(probability)
  i += 1

In [0]:
# now load the data from the dictonary into the dataframe (as column "landmarks")
test_df["landmarks"] = test_df["id"].map(pred_to_filename)
test_df

In [0]:
# assign it to the submission_df dataframe but without the url column
submission_df = test_df.drop(['url'], axis=1).set_index("id")
submission_df

In [0]:
submission_fname = "/your-submssion.csv"

In [0]:
# save the submissiops df as csv to google drive
submission_df.to_csv(path+submission_fname, sep=",", encoding="utf-8")

In [0]:
# save it in the local colab folder
submission_df.to_csv("/content"+submission_fname, sep=",", encoding="utf-8")

In [0]:
# download the file to your machine 
from google.colab import files
files.download("/content"+submission_fname)